In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from IPython.core.display import display, HTML
import re

In [18]:
TARGET_WINRATE = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
ROOT_PATH = '\\\\CILab_NAS_Data/homes/MMORPG/PCG/GeneratedSample'
METHODS = {
    'Random': 'random',
    'Heuristic': 'heuristic',
    'RL': 'heuristic',
}



In [19]:
dfs = []

for win_rate in TARGET_WINRATE:
    for method_name, method_val in METHODS.items():
        file_path = f"{ROOT_PATH}/SkillLog_pcg_winRate-{win_rate}-1.0_{method_val}.csv"
        df = pd.read_csv(file_path)
        df = df.groupby('EpisodeCount').last()
        df.loc[:, 'Method'] = method_name
        df.loc[:, 'Target_WinRate'] = win_rate
        dfs.append(df)

df = pd.concat(dfs)
df['Error_WinRate'] = df.apply(lambda x: abs(x['Result[0]'] - x['Target_WinRate']), axis=1)
# df.drop(df[df.StepCount < 5].index, inplace=True)
origin_df = df.copy()

print(df.describe())
print(len(df))

df['StepCount'].hist()


FileNotFoundError: [Errno 2] No such file or directory: '\\\\CILab_NAS_Data/homes/MMORPG/PCG/GeneratedSample/SkillLog_pcg_winRate-0.1-1.0_random.csv'

In [ ]:
CONTENT_INDEX = list(range(0, 20))
CONTENT_INDEX

In [ ]:
train_x = df[[f'Content[{idx}]' for idx in CONTENT_INDEX]]
tsne = TSNE(n_components=1).fit_transform(train_x)
tsne_df = pd.DataFrame({'axis-0': tsne[:, 0], 'Method': df['Method']})

plt.figure(figsize=(8, 6))
sns.scatterplot(
    x = 'axis-0',
    # y = 'axis-1',
    hue = 'Method',
    palette = sns.color_palette("Set1", 3),
    data = tsne_df,
    legend = "full",
)

plt.title("Generated Skill Distribution")
plt.show()

In [ ]:
df.groupby(['Target_WinRate', 'Method'])['Method', 'Error_WinRate', 'Result[0]'].agg({'Result[0]':['mean','std'],
                                                                         'Error_WinRate':['mean','std']})



pd.options.display.float_format = '{:,.2f}'.format
import numpy as np
from matplotlib import cm
top = cm.get_cmap('Oranges_r', 512)
bottom = cm.get_cmap('Blues', 512)
newcolors = np.vstack((top(np.linspace(0, 1, 512)),
                       bottom(np.linspace(0, 1, 512))))
newcolors = newcolors[:, :3]

MIN = 0
MAX = 1


cmap = plt.cm.get_cmap('RdBu_r')

def rgb_to_hex(r, g, b):

    r, g, b = int(r * 255), int(g * 255), int(b * 255)
    return hex(r)[2:].zfill(2) + hex(g)[2:].zfill(2) + hex(b)[2:].zfill(2)

def styler_2(col):

    cmap = plt.cm.get_cmap('RdBu_r')
    means = col.groupby(level=0).transform('min')

    if col.name[0] != 'Result[0]':

        text_bold = ((col <= means).map({
            True: 'textbf:--rwrap;',
            False: ''
        })).to_list()
    else:
        text_bold = ['' for _ in col]

    colors = list()
    for row, bf_str in zip(col, text_bold):

        font_color = 'white' if abs(row) > 0.4 else 'black'

        _val = row - 0.15
        _val = _val + 0.5
        hex_code = rgb_to_hex(*list(cmap(_val))[:3])

        tag = 'cellcolor:[HTML]{' + hex_code + '}; color:{' + font_color + '}; ' + bf_str
        colors.append(tag)
    return colors

error_result = df.groupby(['Target_WinRate', 'Method'])['Method', 'Error_WinRate', 'Result[0]'].agg({'Result[0]':['mean','std'],
                                                                         'Error_WinRate':['mean','std']})
styled_table = error_result.style.apply(styler_2).format(precision=3)

def concat_mean_std(latex_rows, start_row_num, end_row_num, column_num):

    for row_num in range(start_row_num, end_row_num):
        columns = latex_rows[row_num].split('&')
        first_column = columns[column_num]
        second_column = columns[column_num + 1]

        def _find_value(val):
            pattern = r"(\d+\.\d+)"

            match = re.search(pattern, val)

            extracted_float = match.group()
            return extracted_float

        first_val = _find_value(first_column)
        second_val = _find_value(second_column)

        if second_column.find('\\\\') != -1:
            row_break = ' \\\\'
        else:
            row_break = ''

        first_column = first_column.replace(first_val, '{' + first_val + '} ($\pm{}$' + second_val + ')')

        output = "\multicolumn{2}{c}{%s} %s" % (first_column, row_break)


        columns[column_num] = output
        del columns[column_num + 1]

        latex_rows[row_num] = '&'.join(columns)
    return latex_rows

with open('skill_controllable_table.tex', 'w') as f:
    latex_str = styled_table.to_latex(position_float='centering',
                                      multicol_align="c",
                                      hrules=True)
    latex_str = latex_str.replace('Target_WinRate', '\\begin{tabular}[x]{@{}c@{}}Target\\\\($W_{t}$)\end{tabular}')
    latex_str = latex_str.replace('Error_WinRate', 'RMSE (\|$W_{t}-W_{c}$\|, $\downarrow{}$)')
    latex_str = latex_str.replace('{mean} & {std}', '\multicolumn{2}{c}{Mean ($\pm{}$SD)}')
    latex_str = latex_str.replace('Result[0]', 'Generated ($W_{c}$)')
    latex_str = latex_str.replace('llrrrr', 'cl|cc|cc')
    latex_str = latex_str.replace('\\begin{table}', '\\begin{table}[!h]')
    latex_list = latex_str.splitlines()

    latex_list = concat_mean_std(latex_list, start_row_num=10, end_row_num=25, column_num=2)
    latex_list = concat_mean_std(latex_list, start_row_num=10, end_row_num=25, column_num=3)

    i = 12
    while i < len(latex_list) - 4:
        latex_list.insert(i, '\midrule')
        i += 4

    latex_str = '\n'.join(latex_list)

    f.write(latex_str)


In [ ]:
skill_df = origin_df.copy()
'''
_lastGeneratedSkill[7] += rangeTick * act[0];
_lastGeneratedSkill[8] += coolTick * act[1];
_lastGeneratedSkill[9] += castTimeTick * act[2];
_lastGeneratedSkill[15] += valueTick * act[3];
'''

skill_df = skill_df[["StepCount", "Method", "Target_WinRate",
                       "Content[7]", "Content[8]", "Content[9]", "Content[15]"]]
skill_df.rename(columns = {"Content[7]": "Range", "Content[8]": "Cool Time", "Content[9]": "Cast Time", "Content[15]": "Damage"}, inplace = True)

tsne_np = TSNE(n_components=1).fit_transform(skill_df[["Range", "Cool Time", "Cast Time", "Damage"]])
pca_np = TSNE(n_components=3).fit_transform(skill_df[["Range", "Cool Time", "Cast Time", "Damage"]])

skill_df['t-SNE'] = tsne_np.flatten()
skill_df = skill_df.reset_index()

tsne_df = skill_df.copy()

In [ ]:
import numpy as np
from matplotlib import cm
top = cm.get_cmap('Oranges_r', 512)
bottom = cm.get_cmap('Blues', 512)
newcolors = np.vstack((top(np.linspace(0, 1, 512)),
                       bottom(np.linspace(0, 1, 512))))
newcolors = newcolors[:, :3]

MIN = 0
MAX = 1



cmap = plt.cm.get_cmap('RdBu_r')

def rgb_to_hex(r, g, b):

    r, g, b = int(r * 255), int(g * 255), int(b * 255)
    return hex(r)[2:].zfill(2) + hex(g)[2:].zfill(2) + hex(b)[2:].zfill(2)


color_map = {
    'Range': plt.cm.get_cmap('Blues'),
    'Cool Time': plt.cm.get_cmap('Oranges'),
    'Cast Time': plt.cm.get_cmap('Greens'),
    'Damage': plt.cm.get_cmap('Reds'),
    't-SNE': plt.cm.get_cmap('Greys'),
}


def styler_3(col):

    means = col.groupby(level=0).transform('mean')
    cmap = color_map[col.name[0]]

    min_val, max_val = min(col), max(col)

    def _minmax_scaler(val):
        return (val - min_val) / (max_val - min_val)

    text_bold = ((col >= means).map({
        True: 'textbf:--rwrap;',
        False: ''
    })).to_list()

    colors = list()
    for row, bf_str in zip(col, text_bold):
        scaled = _minmax_scaler(row) * 0.4
        hex_code = rgb_to_hex(*list(cmap(scaled))[:3])
        font_color = 'white' if abs(scaled) > 0.4 else 'black'
        tag = 'cellcolor:[HTML]{' + hex_code + '}; color:{' + font_color + '}; ' + bf_str
        colors.append(tag)

    return colors

result = tsne_df.groupby(['Target_WinRate', 'Method'])['Method', 'Range', 'Cool Time', 'Cast Time', 'Damage', 't-SNE'].agg(
    {'Range':['std'], 'Cool Time':['std'], 'Cast Time':['std'], 'Damage':['std'], 't-SNE':['std']})

styled_table = result.style.apply(styler_3).format(precision=2)

with open('skill_diversity_small.tex', 'w') as f:
    latex_str = styled_table.to_latex(position_float='centering',
                                      multicol_align="c",
                                      hrules=True)
    latex_str = latex_str.replace('Target_WinRate', '\\begin{tabular}[x]{@{}c@{}}Target\\\\($W_{t}$)\end{tabular}')
    latex_str = latex_str.replace('llrrrrr', 'cl|cccc|c')

    latex_str = latex_str.replace('Cool Time', '\\begin{tabular}[x]{@{}c@{}}Cool\\\\Time\end{tabular}')
    latex_str = latex_str.replace('Cast Time', '\\begin{tabular}[x]{@{}c@{}}Cast\\\\Time\end{tabular}')
    latex_str = latex_str.replace('{std}', '{($\pm$SD)}')
    latex_str = latex_str.replace('\\begin{table}', '\\begin{table}[!h]')
    # latex_str = latex_str.replace('Heuristic', 'HR')
    # latex_str = latex_str.replace('Random', 'RD')
    latex_list = latex_str.splitlines()

    i = 11
    while i < len(latex_list) - 4:
        latex_list.insert(i, '\midrule')
        i += 4


    latex_str = '\n'.join(latex_list)
    f.write(latex_str)

result

In [ ]:
c_skill_df = skill_df.copy()
c_skill_df = pd.melt(c_skill_df, id_vars=['EpisodeCount', 'StepCount', 'Method', 'Target_WinRate'],
                             var_name='Parameter', value_name='Value')
c_skill_df

In [ ]:
c_skill_df[c_skill_df["Target_WinRate"] == 0.1]

In [ ]:
import numpy as np
sns.set_theme(style="whitegrid")

x_jitter = np.random.normal(0, 0.3, size=len(c_skill_df))
y_jitter = np.random.normal(0, 0.3, size=len(c_skill_df))

f, ax = plt.subplots(figsize=(6, 4))
clarity_ranking = ["PPO", "Heuristic", "Random"]
sns.boxplot(x="Parameter", y="Value",
                hue="Method",
                # size=None,
                # palette="ch:r=-.2,d=.3_r",
                # inner="quart",
                hue_order=clarity_ranking,
                # sizes=(1, 8),
                # linewidth=0,
                # alpha=0.1,
                data=c_skill_df[c_skill_df["Target_WinRate"] == 0.1], ax=ax)
sns.despine(left=True)


In [ ]:
skill_df = origin_df.copy()
'''
_lastGeneratedSkill[7] += rangeTick * act[0];
_lastGeneratedSkill[8] += coolTick * act[1];
_lastGeneratedSkill[9] += castTimeTick * act[2];
_lastGeneratedSkill[15] += valueTick * act[3];
'''

skill_df = skill_df[["StepCount", "Method", "Target_WinRate", "Error_WinRate",
                       "Content[7]", "Content[8]", "Content[9]", "Content[15]"]]
skill_df.rename(columns = {"Content[7]": "Range", "Content[8]": "Cool Time", "Content[9]": "Cast Time", "Content[15]": "Damage"}, inplace = True)


skill_df = skill_df[(skill_df["Error_WinRate"] < 0.1)]
skill_df = skill_df[(skill_df["Method"] == "RL")]
tsne_np = TSNE(n_components=3).fit_transform(skill_df[["Range", "Cool Time", "Cast Time", "Damage"]])
pca_np = PCA(n_components=3).fit_transform(skill_df[["Range", "Cool Time", "Cast Time", "Damage"]])


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(6, 7))
ax = fig.add_subplot(projection='3d')

palette = sns.color_palette("bright", 7)
TARGET_WINRATE = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

for i, item in enumerate(TARGET_WINRATE):
    print(item)
    c = palette[i]

    index = skill_df["Target_WinRate"] == item

    ax.scatter(tsne_np[index,0], tsne_np[index,1], tsne_np[index,2], s=5.0, marker='o', c=c)


# sns.scatterplot(tsne_np[:,0], tsne_np[:,1], tsne_np[:,2],
#                 hue=skill_df.Target_WinRate,
#                 legend='full',
#                 palette=palette,
#                 style=skill_df.Method)
plt.show()

In [ ]:
corr_df = origin_df.copy()
'''
_lastGeneratedSkill[7] += rangeTick * act[0];
_lastGeneratedSkill[8] += coolTick * act[1];
_lastGeneratedSkill[9] += castTimeTick * act[2];
_lastGeneratedSkill[15] += valueTick * act[3];
'''

corr_df = corr_df[["StepCount", "Method", "Target_WinRate", "Error_WinRate",
                       "Content[7]", "Content[8]", "Content[9]", "Content[15]"]]

corr_df.rename(columns = {"Content[7]": "Range", "Content[8]": "Cool Time", "Content[9]": "Cast Time", "Content[15]": "Damage"}, inplace = True)
corr_df = corr_df[(corr_df["Method"] == "Heuristic")]


# 그림 사이즈 지정
fig, ax = plt.subplots( figsize=(7,7) )

# 삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래 삼각형에 False)
mask = np.zeros_like(corr_df.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# 히트맵을 그린다
sns.heatmap(df,
            cmap = 'RdYlBu_r',
            annot = True,   # 실제 값을 표시한다
            mask=mask,      # 표시하지 않을 마스크 부분을 지정한다
            linewidths=.5,  # 경계면 실선으로 구분하기
            cbar_kws={"shrink": .5},# 컬러바 크기 절반으로 줄이기
            vmin = -1,vmax = 1   # 컬러바 범위 -1 ~ 1
           )
plt.show()

In [ ]:
corr_df = origin_df.copy()
corr_df = corr_df[["StepCount", "Method", "Target_WinRate", "Error_WinRate",
                       "Content[7]", "Content[8]", "Content[9]", "Content[15]"]]
corr_df.rename(columns = {"Content[7]": "Range", "Content[8]": "Cool Time", "Content[9]": "Cast Time", "Content[15]": "Damage"}, inplace = True)


tmp_dict = {'Method': [], 'Target': [], 'Range': [], 'Cool Time': [], 'Cast Time': [], 'Damage': []}

for method in METHODS:
    for win_rate in TARGET_WINRATE:
        _df = corr_df[(corr_df["Target_WinRate"] == win_rate) & (corr_df["Method"] == method)]
        _df_dict = _df.corr()['Error_WinRate'].to_dict()

        tmp_dict['Method'].append(method)
        tmp_dict['Target'].append(win_rate)
        tmp_dict['Range'].append(_df_dict['Range'])
        tmp_dict['Cool Time'].append(_df_dict['Cool Time'])
        tmp_dict['Cast Time'].append(_df_dict['Cast Time'])
        tmp_dict['Damage'].append(_df_dict['Damage'])

corr_df = pd.DataFrame(tmp_dict)
corr_df = corr_df.groupby(['Method', 'Target'])['Range', 'Cool Time', 'Cast Time', 'Damage'].mean()
print(corr_df)


cmap = plt.cm.get_cmap('RdBu_r')

def rgb_to_hex(r, g, b):
    r, g, b = int(r * 255), int(g * 255), int(b * 255)
    return hex(r)[2:].zfill(2) + hex(g)[2:].zfill(2) + hex(b)[2:].zfill(2)

def styler_2(val):
    font_color = 'white' if abs(val) > 0.4 else 'black'
    val = val + 0.5
    return 'cellcolor:[HTML]{' + rgb_to_hex(*list(cmap(val))[:3]) + '}; color:{' + font_color + '};'

styled_table = corr_df.style.applymap(styler_2).format(precision=2)

with open('skill_correlation.tex', 'w') as f:
    latex_str = styled_table.to_latex(position_float='centering',
                                      multicol_align="c",
                                      hrules=True)
    latex_str = latex_str.replace('llrrrr', 'lc|cccc')
    latex_list = latex_str.splitlines()

    i = 14
    while i < len(latex_list) - 7:
        latex_list.insert(i, '\midrule')
        i += 8

    latex_str = '\n'.join(latex_list)

    f.write(latex_str)